In [7]:
import pprint as pp
import requests
import pandas as pd
import requests

In [8]:
# import glob
# import pandas as pd

# # 设置您的文件夹路径
# folder_path = "../coding_problems/rainfall"

# # 使用glob模块来获取文件夹下所有的csv文件
# csv_files = glob.glob(folder_path + "/*.csv")

# # 创建一个空的列表用来存储所有数据
# all_data = []

# # 循环遍历所有CSV文件
# for filename in csv_files:
#     # 读取每个CSV文件
#     # 找到'avgtmp.csv'之前的部分
#     end_index = filename.rfind("rain.csv")

#     # 找到最后一个'\'的位置
#     start_index = filename.rfind("\\", 0, end_index)

#     # 截取两者之间的字符串作为newName
#     newName = filename[start_index + 1 : end_index]
#     df = pd.read_csv(filename, skiprows=4)
#     # 将"Date"列转换为日期格式，并设置为索引
#     df["Date"] = pd.to_datetime(df["Date"], format="%Y%m")
#     df.set_index("Date", inplace=True)

#     # 按季度重新采样数据，并计算每个季度的平均值
#     df_resampled = df.resample("Q").mean()

#     # 创建一个新的DataFrame，其中每个季度的平均值是一个新的列
#     row = pd.DataFrame(
#         {newName: df_resampled["Value"].values},
#         index=df_resampled.index.to_period("Q").astype(str),
#     )

#     # 将读取的数据追加到all_data列表
#     all_data.append(row.T)

# # 使用concat函数合并所有的数据
# all_data = pd.concat(all_data, axis=0)

# # 修改第一列的列名为'State'
# all_data.rename_axis("State", axis=0, inplace=True)
# all_data.rename_axis(None, axis=1, inplace=True)
# # 重置索引并将新列命名为'State'
# all_data.reset_index(inplace=True)
# all_data.rename(columns={"index": "State"}, inplace=True)


# # all_data现在包含了所有CSV文件的数据
# all_data["State"]

In [9]:
# all_data.to_csv("PrecipitationByStates.csv", index=False)
# all_data.to_csv("TemperatureByStates.csv", index=False)

In [10]:
def get_latest_events():
    try:
        response = requests.get("https://eonet.gsfc.nasa.gov/api/v3/events")
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def preprocess_wildfires_df(events):
    events = get_latest_events()
    if events:
        # Extract relevant information from each event and add it to a list
        data = []
        for event in events["events"]:
            data.append(
                {
                    "Event": event["title"],
                    "Categories": event["categories"][0]["title"]
                    if event["categories"]
                    else None,
                    "Date": event["geometry"][0]["date"] if event["geometry"] else None,
                    "Location": event["geometry"][0]["coordinates"]
                    if event["geometry"]
                    else None,
                }
            )
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(data)
        df["Country"] = df["Event"].apply(
            lambda event: "United States"
            if "States" in event
            else event.split(",")[-1].strip()
        )
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Season"] = df["Date"].dt.to_period("Q").astype(str)

        df[["Longitude", "Latitude"]] = pd.DataFrame(
            df["Location"].tolist(), index=df.index
        )

    df = df[(df["Country"] == "United States") & (df["Categories"] == "Wildfires")]

    return df

In [11]:
events = get_latest_events()
df = preprocess_wildfires_df(events)

C:\Users\redhawc\AppData\Local\Temp\ipykernel_25332\3915679507.py:41: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [12]:
df.head()

,Event,Categories,Date,Location,Country,Year,Season,Longitude,Latitude
22,"Major Fire, Ocala National Forest, Florida, Un...",Wildfires,2023-04-27 14:15:00+00:00,"[-81.688889, 29.229722]",United States,2023,2023Q2,-81.688889,29.229722
55,"Garceau Fire, W of Polson, Montana, United States",Wildfires,2022-08-16 00:00:00+00:00,"[-114.489, 47.694]",United States,2022,2022Q3,-114.489000,47.694000
105,"Wildfire - N of Merced, California - United St...",Wildfires,2022-02-16 20:44:00+00:00,"[-120.533005422, 37.543469882]",United States,2022,2022Q1,-120.533005,37.543470
106,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-02-13 20:40:00+00:00,"[-110.862229905, 33.417091555]",United States,2022,2022Q1,-110.862230,33.417092
111,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-01-28 08:38:00+00:00,"[-110.860071422, 33.416654251]",United States,2022,2022Q1,-110.860071,33.416654


In [13]:
# !pip install plotly geopandas pyshp shapely

In [14]:
# import pandas as pd

# us_cities = pd.read_csv(
#     "https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv"
# )

# import plotly.express as px

# fig = px.scatter_mapbox(
#     df,
#     lat="Latitude",
#     lon="Longitude",
#     hover_name="Date",
#     hover_data=["Date", "Event"],
#     color_discrete_sequence=["fuchsia"],
#     zoom=3,
#     height=300,
# )
# fig.update_layout(
#     mapbox_style="white-bg",
#     mapbox_layers=[
#         {
#             "below": "traces",
#             "sourcetype": "raster",
#             "sourceattribution": "United States Geological Survey",
#             "source": [
#                 "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
#             ],
#         },
#         {
#             "sourcetype": "raster",
#             "sourceattribution": "Government of Canada",
#             "source": [
#                 "https://geo.weather.gc.ca/geomet/?"
#                 "SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX={bbox-epsg-3857}&CRS=EPSG:3857"
#                 "&WIDTH=1000&HEIGHT=1000&LAYERS=RADAR_1KM_RDBR&TILED=true&FORMAT=image/png"
#             ],
#         },
#     ],
# )
# fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# fig.show()

In [15]:
# from urllib.request import urlopen
# import json

# with urlopen(
#     "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
# ) as response:
#     counties = json.load(response)

# import pandas as pd

# df = pd.read_csv(
#     "https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#     dtype={"fips": str},
# )

# import plotly.graph_objects as go

# fig = go.Figure(
#     go.Choroplethmapbox(
#         geojson=counties,
#         locations=df.fips,
#         z=df.unemp,
#         colorscale="Viridis",
#         zmin=0,
#         zmax=12,
#         marker_opacity=0.5,
#         marker_line_width=0,
#     )
# )
# fig.update_layout(
#     mapbox_style="carto-positron",
#     mapbox_zoom=3,
#     mapbox_center={"lat": 37.0902, "lon": -95.7129},
# )
# fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# fig.show()

In [16]:
temperature = pd.read_csv("./TemperatureByStates_Updated.csv")
precipitation = pd.read_csv("./PrecipitationByStates_Updated.csv")

In [17]:
precipitation.head()

,State,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,code,state
0,Alabama,5.330000,6.323333,6.583333,3.483333,5.880000,4.620000,4.783333,4.000000,5.556667,4.923333,3.876667,1.40,AL,Alabama
1,Alaska,2.623333,2.260000,4.353333,3.656667,3.463333,1.140000,5.653333,4.073333,2.593333,2.600000,4.526667,4.06,AK,Alaska
2,Arizona,0.756667,0.150000,2.430000,0.853333,0.426667,0.240000,2.320000,1.036667,1.790000,0.190000,1.186667,0.20,AZ,Arizona
3,Arkansas,4.556667,5.713333,3.203333,3.270000,4.520000,5.443333,2.956667,4.310000,6.276667,4.440000,3.593333,4.67,AR,Arkansas
4,California,2.256667,0.180000,0.250000,3.623333,0.580000,0.816667,0.453333,2.846667,6.286667,0.633333,0.466667,0.45,CA,California


In [18]:
df.head()

,Event,Categories,Date,Location,Country,Year,Season,Longitude,Latitude
22,"Major Fire, Ocala National Forest, Florida, Un...",Wildfires,2023-04-27 14:15:00+00:00,"[-81.688889, 29.229722]",United States,2023,2023Q2,-81.688889,29.229722
55,"Garceau Fire, W of Polson, Montana, United States",Wildfires,2022-08-16 00:00:00+00:00,"[-114.489, 47.694]",United States,2022,2022Q3,-114.489000,47.694000
105,"Wildfire - N of Merced, California - United St...",Wildfires,2022-02-16 20:44:00+00:00,"[-120.533005422, 37.543469882]",United States,2022,2022Q1,-120.533005,37.543470
106,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-02-13 20:40:00+00:00,"[-110.862229905, 33.417091555]",United States,2022,2022Q1,-110.862230,33.417092
111,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-01-28 08:38:00+00:00,"[-110.860071422, 33.416654251]",United States,2022,2022Q1,-110.860071,33.416654


In [19]:
# import plotly.graph_objects as go

# temperture["text"] = temperture["state"] + "<br>"


# fig = go.Figure(
#     data=go.Choropleth(
#         locations=temperture["code"],
#         z=temperture["2021Q1"].astype(float),
#         locationmode="USA-states",
#         colorscale="Reds",
#         autocolorscale=False,
#         text=temperture["text"],  # hover text
#         marker_line_color="white",  # line markers between states
#         colorbar_title="↑ Temperature",
#     )
# )

# # Add scatter plot for wildfire data
# fig.add_trace(
#     go.Scattergeo(
#         lon=df["Longitude"],
#         lat=df["Latitude"],
#         text=["🔥" for _ in range(len(df))],  # 使用火焰 emoj
#         mode="text",  # 将模式设置为"text"以显示文本标记
#         textfont=dict(
#             family="Arial",  # 可以选择合适的字体
#             size=12,  # 调整大小以适合你的图表
#             color="red",  # 文本的颜色，这里是红色
#         ),
#     )
# )


# fig.update_layout(
#     title_text="2021-2023 US Wildfire Cases Visualisation<br> Compared With Temperature & Precipitation By State",
#     geo=dict(
#         scope="usa",
#         projection=go.layout.geo.Projection(type="albers usa"),
#         showlakes=True,  # lakes
#         lakecolor="rgb(255, 255, 255)",
#     ),
# )

# fig.show()

In [20]:
# import plotly.graph_objects as go

# # 假设 temperature 是您的温度数据集
# temperature["text"] = temperature["state"] + "<br>"

# # 创建图表
# fig = go.Figure()

# # 添加初始季度的温度数据
# fig.add_trace(
#     go.Choropleth(
#         locations=temperature["code"],
#         z=temperature["2021Q1"].astype(float),  # 初始显示2021Q1数据
#         locationmode="USA-states",
#         colorscale="Reds",
#         autocolorscale=False,
#         text=temperature["text"],  # hover text
#         marker_line_color="white",  # line markers between states
#         colorbar_title="↑ Temperature",
#     )
# )

# # 假设 df 是您的野火数据集
# # 添加野火数据
# fig.add_trace(
#     go.Scattergeo(
#         lon=df["Longitude"],
#         lat=df["Latitude"],
#         text=["🔥" for _ in range(len(df))],  # 使用火焰 emoji
#         mode="text",  # 将模式设置为"text"以显示文本标记
#         textfont=dict(
#             family="Arial",  # 可以选择合适的字体
#             size=12,  # 调整大小以适合你的图表
#             color="red",  # 文本的颜色，这里是红色
#         ),
#     )
# )

# # 更新布局
# fig.update_layout(
#     title_text="2021-2023 US Wildfire Cases Visualisation<br>Compared With Temperature & Precipitation By State",
#     geo=dict(
#         scope="usa",
#         projection=go.layout.geo.Projection(type="albers usa"),
#         showlakes=True,  # lakes
#         lakecolor="rgb(255, 255, 255)",
#     ),
# )

# # 添加滑动条以切换季度
# seasons = [
#     "2021Q1",
#     "2021Q2",
#     "2021Q3",
#     "2021Q4",
#     "2022Q1",
#     "2022Q2",
#     "2022Q3",
#     "2022Q4",
#     "2023Q1",
#     "2023Q2",
#     "2023Q3",
#     "2023Q4",
# ]
# steps = []

# for season in seasons:
#     step = dict(
#         method="update",
#         args=[
#             {"z": [temperature[season].astype(float), None]},
#             {"title": f"Temperature - {season}"},
#         ],
#         label=season,
#     )
#     steps.append(step)

# slider = dict(active=0, currentvalue={"prefix": "Season: "}, steps=steps)

# fig.update_layout(sliders=[slider])

# # 显示图表
# fig.show()

In [21]:
import plotly.graph_objects as go

# 假设 temperature 是您的温度数据集
temperature["text"] = temperature["state"] + "<br>"

# 创建图表
fig = go.Figure()

# 添加初始季度的温度数据轨迹
temperature_trace = go.Choropleth(
    visible=True,  # 保证初始季度的温度数据可见
    locations=temperature["code"],
    z=temperature["2021Q3"].astype(float),  # 初始显示2021Q3
    locationmode="USA-states",
    colorscale="Reds",
    autocolorscale=False,
    text=temperature["text"],  # hover text
    marker_line_color="white",  # line markers between states
    colorbar_title="↑ Temperature",
)
fig.add_trace(temperature_trace)

# 假设 wildfires 是您的野火数据集
# 为每个季度添加野火数据轨迹
seasons = [
    "2021Q1",
    "2021Q2",
    "2021Q3",
    "2021Q4",
    "2022Q1",
    "2022Q2",
    "2022Q3",
    "2022Q4",
    "2023Q1",
    "2023Q2",
    "2023Q3",
    "2023Q4",
]
for season in seasons:
    season_df = df[df["Season"] == season]
    fig.add_trace(
        go.Scattergeo(
            visible=(season == "2021Q3"),  # 仅对初始季度设为可见
            lon=season_df["Longitude"],
            lat=season_df["Latitude"],
            text=["🔥" for _ in range(len(season_df))],  # 使用火焰 emoji
            mode="text",  # 将模式设置为"text"以显示文本标记
            textfont=dict(
                family="Arial",  # 可以选择合适的字体
                size=12,  # 调整大小以适合你的图表
                color="red",  # 文本的颜色，这里是红色
            ),
        )
    )

# 更新布局
fig.update_layout(
    title_text="2021-2023 US Wildfire Cases Visualisation<br>Compared With Temperature & Precipitation By State",
    geo=dict(
        scope="usa",
        projection=go.layout.geo.Projection(type="albers usa"),
        showlakes=True,  # lakes
        lakecolor="rgb(255, 255, 255)",
    ),
)

# 添加滑动条以切换季度
steps = []
for index1, season in enumerate(seasons):
    step = dict(
        method="restyle",
        args=[
            {
                "z": [temperature[season].astype(float)],
                "visible": [True]
                + [index1 == index2 for index2 in range(len(seasons))],
            },
        ],
        label=season,
    )
    steps.append(step)

slider = dict(active=2, currentvalue={"prefix": "Season: "}, steps=steps)

fig.update_layout(sliders=[slider])

# 显示图表
fig.show()

In [22]:
# visualisation(temperture, Precipitation, df)

In [23]:
# !pip install dash


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': '选项 1', 'value': '选项 1'},
            {'label': '选项 2', 'value': '选项 2'},
            {'label': '选项 3', 'value': '选项 3'}
        ],
        value='选项 1'
    ),
    html.Div(id='display-selected-value')
])

@app.callback(
    Output('display-selected-value', 'children'),
    [Input('dropdown', 'value')]
)
def update_output(value):
    return f'您选择了: {value}'

app.run_server(mode='inline')  # 使用内联模式


c:\Users\redhawc\.pyenv\pyenv-win\versions\3.11.2\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



TypeError: 'NoneType' object cannot be interpreted as an integer